In [1]:
!rm -r /kaggle/working/*
%cd /kaggle/working

/kaggle/working


In [2]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)
sys.path.append(os.path.join(PACKAGE_DIR, "Penguin-ML-Library"))

In [3]:
import whoosh_utils_penguin

Processing /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl
Whoosh is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import polars as pl

In [5]:
train_idx = whoosh_utils_penguin.load_index(
    "/kaggle/input/uspto-boolean-search-optimization/train_index"
)
searcher = whoosh_utils_penguin.get_searcher(train_idx)
qp = whoosh_utils_penguin.get_query_parser()

In [6]:
query = "ti:balloons OR ti:string"
print(whoosh_utils_penguin.count_query_tokens(query=query))
results = whoosh_utils_penguin.execute_query(query, qp, searcher)[:5]
results

3


[<Hit {'id': 'US-9407362-B2'}>,
 <Hit {'id': 'US-4098163-A'}>,
 <Hit {'id': 'US-2011056083-A1'}>,
 <Hit {'id': 'US-4984694-A'}>,
 <Hit {'id': 'US-8785276-B2'}>]

In [7]:
os.makedirs("temp", exist_ok=True)
whoosh_utils_penguin.create_index(
    output_dir="temp",
    documents="/kaggle/input/uspto-boolean-search-optimization/patent_data/*.parquet",
    debug=True,
)

indexing: 100%|██████████| 10/10 [00:01<00:00,  7.68it/s]


In [8]:
train_idx = whoosh_utils_penguin.load_index("/kaggle/working/temp")
searcher = whoosh_utils_penguin.get_searcher(train_idx)
qp = whoosh_utils_penguin.get_query_parser()

In [9]:
query = "ti:b*"
print(whoosh_utils_penguin.count_query_tokens(query=query))
results = whoosh_utils_penguin.execute_query(query, qp, searcher)[:5]
results

1


[<Hit {'ab': '', 'clm': '', 'cpc': 'A41B1/00', 'detd': '1955 F. A. ROBINSON COMBINED SHIRTWAIST OR BLOUSE AND SUSPENDERS Filed Jan. 21, 1952 Flornce J7. Robinson INVENTOR ATTORNEYS United States Patent COMBINED SHIRTWAIST OR BLOUSE AND SUSPENDERS Florence A. Robinson, Washington, D. C.  \n Application January 21, 1952, Serial No. 267,329  \n 1 Claim. (Cl. 2-107) This invention relates to wearing apparel, and particularly wearing apparel of the shirtwaist or blouse type, the primary object of the invention being to provide suspenders as forming a part of the garment, the suspenders being so constructed and arranged that they will be invisible at all times, thereby eliminating the unsightly appearance of suspenders worn exteriorly of the garment, as is common practice.  \n  An important object of the invention is to so arrange the suspenders within the garment that the suspenders will receive the strain incident to the supporting of trousers, with which the shirtwaist or blouse is used. 

In [10]:
query = "id:US-2698435-A"
print(whoosh_utils_penguin.count_query_tokens(query=query))
results = whoosh_utils_penguin.execute_query(query, qp, searcher)[:5]
results

1


[<Hit {'ab': '', 'clm': '', 'cpc': 'A41B1/00', 'detd': '1955 F. A. ROBINSON COMBINED SHIRTWAIST OR BLOUSE AND SUSPENDERS Filed Jan. 21, 1952 Flornce J7. Robinson INVENTOR ATTORNEYS United States Patent COMBINED SHIRTWAIST OR BLOUSE AND SUSPENDERS Florence A. Robinson, Washington, D. C.  \n Application January 21, 1952, Serial No. 267,329  \n 1 Claim. (Cl. 2-107) This invention relates to wearing apparel, and particularly wearing apparel of the shirtwaist or blouse type, the primary object of the invention being to provide suspenders as forming a part of the garment, the suspenders being so constructed and arranged that they will be invisible at all times, thereby eliminating the unsightly appearance of suspenders worn exteriorly of the garment, as is common practice.  \n  An important object of the invention is to so arrange the suspenders within the garment that the suspenders will receive the strain incident to the supporting of trousers, with which the shirtwaist or blouse is used. 

In [11]:
metadata = pl.read_parquet(
    "/kaggle/input/uspto-boolean-search-optimization/patent_metadata.parquet"
)
metadata.head(2)

publication_number,publication_date,filing_date,family_id,cpc_codes
str,datetime[μs],datetime[μs],f64,list[str]
"""US-1-A""",1836-07-13 00:00:00,null,2.060279e6,"[""B61C11/04""]"
"""US-1-P""",1931-08-18 00:00:00,1930-08-06 00:00:00,4.9893162e7,"[""A01H5/02"", ""A01H6/749""]"


In [12]:
len(metadata)

13307751

In [13]:
patent_df = pl.read_parquet(
    "/kaggle/input/uspto-boolean-search-optimization/patent_data/2010_7.parquet"
)
patent_df.head(2)

publication_number,title,abstract,claims,description
str,str,str,str,str
"""US-2010162461-…","""Women&#39;s Cl…","""The inventive …","""1 . A close-fi…","""INCORPORATION …"
"""US-2010162462-…","""Fishing Trouse…","""A pair of fish…","""1 . A pair of …","""BACKGROUND OF …"
